<a href="https://colab.research.google.com/github/tua2542/DSLoveyou/blob/master/HousePrices_DSLoveYou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello world")

hello world


In [2]:
print("hello world")

hello world
